# Sección 0: Demostración Web Scraping

El objetivo del proyecto era extraer la información de gerentes históricos y actuales de muchas compañías. Se accedió al portal de Supercias. Extraer la información manualmente tomaba, en promedio, un minuto por observación. Adicionalmente, después de extraer muchas observaciones, el proceso se volvía fatigante y solía haber errores en el guardado de la información. Se optó por utilizar un script de web scraping para extraer la información automáticamente y sin ningún tipo de error humano.

In [3]:
import pandas as pd
# #Leer el archivo con los expedientes de las compañías de las que se quería conocer los gerentes
# #actuales
# import cv2
# import pytesseract
# import os
# from bs4 import BeautifulSoup
# import html5lib
# from selenium.webdriver.support.ui import Select
# import pandas as pd
dframe = pd.read_csv("allexps.csv")
exps = list(set(dframe["expediente"]))[:1000]
len(exps)

1000

In [3]:
#Definición de la función que extrae y guarda la información.
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import pandas as pd
import cv2
import pytesseract
import os
import urllib.request as ur
import html5lib
from bs4 import BeautifulSoup
import os
from urllib.error import HTTPError

def solve(src):
    #Save and Read the captcha image
    ur.urlretrieve(src,"captcha.png")
    #Gaming PC
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    #DHUB PC
#    pytesseract.pytesseract.tesseract_cmd = r"C:\Users\eduranh\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    #Other PC
#     pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Py Tesseract\tesseract.exe"
    img = cv2.imread("captcha.png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    answer = pytesseract.image_to_string(img)
#     print(answer)
    #Remove image after reading it so the next iteration can save a new captcha.
    os.remove("captcha.png")
    return answer
    #Give a response to the captcha
def checkDuplicates(driver):
    path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\Décimosegundo Semestre\Web Scraping Course\Financials"
    files = os.listdir(path)
    for file in files:
        check = "(" in file
        if check == True:
            print(file)
            os.remove(f"{path}\\{file}")
            return check
    
def solve_other(driver):
    answer = 0
    time.sleep(3)
    try:
        captcha = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"frmCaptcha:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)
        bar = driver.find_element(By.ID,"frmCaptcha:captcha")
        time.sleep(1)
        bar.send_keys(answer)
        bar.send_keys(Keys.ENTER)
        print("Captcha Solved")
        return answer
    except:
#         print("No Captcha")
        pass
    return answer
def lookForAll(exps):
    path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\Décimosegundo Semestre\Web Scraping Course"
    url = "https://appscvsconsultas.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf"
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs",{"download.default_directory":path,
                                          "download.prompt_for_download":False,
                                          "download.directory_upgrade":True,
                                          "plugins.always_open_pdf_externally":True})
    driver = webdriver.Chrome(options=options)

    used_rucs = []
    count = 0
    #Empty dataframe (where all the other dataframes will be stored):
    frame = pd.DataFrame()
    while len(exps) > 0:
#         try:
        time.sleep(0.5)
        count += 1
        driver.get(url)
        used = exps.pop(0)
        expedient = driver.find_elements(By.CLASS_NAME,
                                         "ui-autocomplete-input.ui-inputfield.ui-widget.ui-state-default.ui-corner-all"
                                        )
        expedient[0].send_keys(
            str(used)
        )
        time.sleep(1)
        expedient[0].send_keys(Keys.ENTER)
        time.sleep(1)
        #Find the Capthca image
        captcha = driver.find_element(By.ID,"frmBusquedaCompanias:captchaImage")
        src = captcha.get_attribute("src")
        answer = solve(src)

        capi = driver.find_element(By.CSS_SELECTOR,
            "input.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
        capi.send_keys(answer)
        time.sleep(0.5)

        #Click Yearly Information Tab:
        btn = driver.find_element(By.ID,"frmMenu:menuInformacionAnualPresentada")
        btn.click()
        #Find the Historical Balance sheets submitted by the company
        box = driver.find_element(By.ID,"frmInformacionCompanias:tblInformacionAnual:j_idt587:filter")
        box.send_keys("3.1.1")

        #In case captcha comes up, solve it:
        solve_other(driver)
        time.sleep(1.5)
        elements = Select(driver.find_element(By.CSS_SELECTOR,
                               "select.ui-paginator-rpp-options.ui-widget.ui-state-default.ui-corner-left"))
        elements.select_by_visible_text("*")

        #Save page source to find stuff with BS4:
        source2 = driver.page_source

        #Use stored page_source1 to find the pertinent table:
        soup = BeautifulSoup(source1,"lxml")
        #Isolate the table we want to grab information from:
        table = soup.find("div",class_="ui-datatable-tablewrapper")
        #Convert this table to a dataframe:
        current = pd.read_html(str(table))[0]
        current.insert(0,"Expediente",used)

        #Use the second stored page_source to find the pertinent table
        soup2 = BeautifulSoup(source2,"lxml")
        #Grab the table and save it as dataframe.
        table2 = soup2.find("div",class_ = "ui-datatable-tablewrapper")
        previous = pd.read_html(str(table2))[0]
        #Clear away the Filter by from the column names:
        for i in previous:
            if "Filter by" in i:
                word = i.split(" ")[-1]
                previous.rename(columns = {i:i.replace(f"Filter by {word}","")},inplace = True)
        previous.insert(0,"Expediente",used)

        #Join the Created Dataframes
        frame = pd.concat([frame,current,previous]).reset_index(drop = True)
#         except:
#             exps.insert(0,used)
#             continue
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
    display(frame)
    return driver,frame

# Placeholder: Repurposing of the scraping script

### Part 1: Navigation Function.

This navFin function records all the navigation steps until we can interact with the PDFs of the financial information. We express the thing as a function, because we might need to reutilize it if a PDF doesn't load properly. Plus it makes the final script look cleaner.

In [4]:
""" from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import pandas as pd
import cv2
import pytesseract
import os
import urllib.request as ur
import html5lib
from bs4 import BeautifulSoup
import os
from urllib.error import HTTPError

def navFin(driver):
    try:
        #Click Yearly Information Tab:
        btn = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((
                By.ID,"frmMenu:menuInformacionAnualPresentada")))
        btn.click()
        time.sleep(2)
        #In case captcha comes up, solve it:
        solve_other(driver)
        print("Financial Info Opened Successfully")
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException):
        print("Nav. to financial failed")
        driver.refresh()
        navFin(driver)


def pdfShow(driver):
    time.sleep(5)
#     try:
        #Find the Historical Balance sheets submitted by the company
#     box = WebDriverWait(driver,10).until(EC.element_to_be_clickable((
#         By.CLASS_NAME,"fui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")))
    #Seach for Balance Sheets (Only identifiable by class name. Second object with that name)
    wait = WebDriverWait(driver,10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME,"ui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")))
    driver.find_elements(
        By.CLASS_NAME,
        "ui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")[1].send_keys("3.1.1")
    print("Keys Sent")
    #Solve Captcha if it pops up
    time.sleep(0.5)
    solve_other(driver)
    time.sleep(0.5)
    #Show every document ever uploaded.
    elements = Select(driver.find_element(By.CSS_SELECTOR,
                           "select.ui-paginator-rpp-options.ui-widget.ui-state-default.ui-corner-left"))
    elements.select_by_visible_text("*")
    time.sleep(2)
#     except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException) as e:
#         print("PDF Showing Failed")
#         print(e)
#         driver.refresh()
#         navFin(driver)
#         time.sleep(5)
#         pdfShow(driver)
def join(rows_u):
    d = []
    for i in rows_u:
        d.extend(i)
    return d

def download(driver,count = 0):
    try:
        print("Beginning Navigation")
        navFin(driver)
        time.sleep(3)
        pdfShow(driver)
        print("Navigation Finished")
        #Fake rows element to start while loop
        rows = range(100)
        #Set an action object to execute clicks on the webpage at will.
        action = ActionChains(driver)
        #Loop until every single file has been downloaded.
        while len(rows[count:]) > 0:
            print(count)
            #Rows where the PDFs are:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME,"ui-widget-content.ui-datatable-even")))
            rows_p = driver.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")
            rows_i = driver.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-odd")
            rows_u = zip(rows_p,rows_i)
            rows = join(rows_u)

            #Open the desired PDF.
    #         wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"ui-widget-content.ui-datatable-even")))
            pdf = rows[count].find_element(By.CLASS_NAME,"ui-commandlink.ui-widget")
            driver.execute_script("arguments[0].click();",pdf)
            #Solve Captha if it pops up
            time.sleep(0.5)
            solve_other(driver)
            time.sleep(2)
            print("Starting Download")
            #Download the files.
            wait.until(EC.element_to_be_clickable((By.ID,"dlgPresentarDocumentoPdf")))
            x = driver.find_element(By.ID,"dlgPresentarDocumentoPdf")
            if count == 0:
                print("First Download")
        #         Download Button (First Download)
        
                action.move_to_element_with_offset(x,370,-230)
                action.click()
                action.perform()
                time.sleep(3)
            else:
                #Download Button (After first download)
                action.move_to_element_with_offset(x,365,-200)
                action.click()
                action.perform()
           #X Button (After first download)
            action.move_to_element_with_offset(x,470,-250)
            action.click()
            action.perform()
            count += 1
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError) as e:
        print("Load Failed...")
        driver.refresh()
        download(driver,count)
    except NoSuchElementException:
        print("No PDF available for this year")
        count += 1
        download(driver,count)
    except IndexError:
        print("No more PDFs") """

In [28]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,NoSuchElementException,UnexpectedAlertPresentException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import pandas as pd
import cv2
import pytesseract
import os
import urllib.request as ur
import html5lib
from bs4 import BeautifulSoup
import os
from urllib.error import HTTPError

#Create a custom error so I can use it whenever I want the program to fail
class PlannedException(Exception):
    "Exception for whenever I need a part of the program to fail"
    pass

def update():
    with open("allexps.csv") as f:
        content = f.readlines()
    a = os.listdir("Financials")
    content.remove("expediente\n")
    for i in a:
        content.remove(f"{i}\n")
    for i in range(len(content)):
        content[i] = content[i].replace("\n","")
    return content

def solve(src):
    #Save and Read the captcha image
    ur.urlretrieve(src,"captcha.png")
    #Gaming PC
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    #DHUB PC
#    pytesseract.pytesseract.tesseract_cmd = r"C:\Users\eduranh\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    #Other PC
#     pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Py Tesseract\tesseract.exe"
    img = cv2.imread("captcha.png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    answer = pytesseract.image_to_string(img)
#     print(answer)
    #Remove image after reading it so the next iteration can save a new captcha.
    os.remove("captcha.png")
    return answer

def solve_other(driver):
    answer = 0
    time.sleep(3)
    try:
        captcha = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"frmCaptcha:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)
        bar = driver.find_element(By.ID,"frmCaptcha:captcha")
        time.sleep(1)
        bar.send_keys(answer)
        bar.send_keys(Keys.ENTER)
        print("Captcha Solved")
        return answer
    except:
#         print("No Captcha")
        pass
    return answer

def waitDownload(p):
    print("Waiting for download",end="")
    while any([file.endswith("crdownload") for file in os.listdir(p)]):
        print(".",end="")
        time.sleep(0.5)

def navFin(driver,fails = 0):
    try:
        #Click Yearly Information Tab:
        btn = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((
                By.ID,"frmMenu:menuDocumentacion")))
        btn.click()

        #In case captcha comes up, solve it:
        time.sleep(1)
        solve_other(driver)
        time.sleep(1)

        #Open the Economic Documents tab
        driver.find_elements(By.CLASS_NAME,"ui-tabs-header.ui-state-default.ui-corner-top")[2].click()
  
  #In case captcha comes up, solve it:
        time.sleep(1)
        solve_other(driver)
        time.sleep(1)

        print("Financial Info Opened Successfully")
        fails = 0
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException):
        print("Nav. to financial failed")
        fails += 1
        driver.refresh()
        if fails < 10:
            navFin(driver)
        else:
            raise PlannedException("Too many failures.")
            


def pdfShow(driver):
    time.sleep(5)
    wait = WebDriverWait(driver,10,0.2)
    wait.until(EC.element_to_be_clickable((By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")))
    #Find the adequate information table.
    m_table = driver.find_element(By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")
    #Find the pertinent search bar with the proper search criteria.
    s_bar = m_table.find_element(By.CLASS_NAME,"ui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
    s_bar.send_keys("AUD")

    #Audited Financials won't always be available. If it's the case, then extract every statement separately.
    #Wait until the table updates to check if there are any elements to download.
    time.sleep(5)
    wait.until(EC.element_to_be_clickable(s_bar))


    print(len(m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")))
    if len(m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")) == 0:
        print("No audited financial statements found. Resorting to downloading the regular financials")
        s_bar.send_keys(Keys.CONTROL + "a")
        s_bar.send_keys(Keys.DELETE)
        time.sleep(0.5)
        wait.until(EC.element_to_be_clickable(s_bar))
        s_bar.send_keys("Estado")

    #Solve Captcha if it pops up
    time.sleep(0.5)
    solve_other(driver)
    time.sleep(0.5)

    #Show every document ever uploaded.
    elements = Select(m_table.find_element(By.CSS_SELECTOR,
                           "select.ui-paginator-rpp-options.ui-widget.ui-state-default.ui-corner-left"))
    elements.select_by_visible_text("*")
 
def join(rows_u):
    d = []
    for i in rows_u:
        d.extend(i)
    return d

def download(driver,p,count = 0):
    fails = 0
    try:
        wait = WebDriverWait(driver,10)
        print("Beginning Navigation")
        navFin(driver)
        time.sleep(3)
        pdfShow(driver)
        print("Navigation Finished")
        #Fake rows element to start while loop
        rows = range(100)
        #Set an action object to execute clicks on the webpage at will.
        action = ActionChains(driver)
        #Loop until every single file has been downloaded.
        while len(rows[count:]) > 0:
            file_num = len(os.listdir(p))
            print(f"Current amount of files in dir: {file_num}")
            print(f"File number {count}")
            #Rows where the PDFs are:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME,"ui-widget-content.ui-datatable-even")))
            rows_p = driver.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")
            rows_i = driver.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-odd")
            rows_u = zip(rows_p,rows_i)
            rows = join(rows_u)

            #Open the desired PDF.
    #         wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"ui-widget-content.ui-datatable-even")))
            pdf = rows[count].find_element(By.CLASS_NAME,"ui-commandlink.ui-widget")
            driver.execute_script("arguments[0].click();",pdf)
            #Solve Captha if it pops up
            time.sleep(0.5)
            solve_other(driver)
            time.sleep(2)
            print("Starting Download")
            #Download the files.
            wait.until(EC.element_to_be_clickable((By.ID,"dlgPresentarDocumentoPdf")))
            x = driver.find_element(By.ID,"dlgPresentarDocumentoPdf")
            if count == 0:
                print("First Download")
        #         Download Button (First Download)
                action.move_to_element_with_offset(x,365,-290)
                action.click()
                action.perform()
                time.sleep(3)
            else:
                #Download Button (After first download)
                action.move_to_element_with_offset(x,365,-270)
                action.click()
                action.perform()
           #X Button (After first download)
            action.move_to_element_with_offset(x,470,-320)
            action.click()
            action.perform()
            #Check if file was actually downloaded
            print(f"New amount of files in dir: {len(os.listdir(p))}")
            if file_num == len(os.listdir(p)):
                print("No file was downloaded. Repeating iteration...")
                driver.refresh()
                download(driver,p,count)
                fails += 1
                if fails > 10:
                    raise PlannedException("Documents consistently not downloading. Starting again.")
            count += 1      
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,PlannedException) as e:
        print("Load Failed...")
        driver.refresh()
        download(driver,p,count)
    except NoSuchElementException:
        print("No PDF available for this year")
        count += 1
        download(driver,p,count)
    except IndexError:
        print("No more PDFs")

### Testing the Captcha Solver

In [27]:
""" captcha = wait.until(EC.presence_of_element_located((By.ID,"frmBusquedaCompanias:captchaImage")))
src = captcha.get_attribute("src") """
answer = solve(src)
answer

'379447\n'

### Extraction Script

In [29]:
path = r"D:\Usuarios\Edwin\OneDrive - Universidad San Francisco de Quito\USFQ Online\DataHub\Supercias\Financial Information\Financials"
# path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\DataHub\Supercias\Financial Information\Financials"
url = "https://appscvsconsultas.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf"
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs",{"download.default_directory":path,
                                      "download.prompt_for_download":False,
                                      "download.directory_upgrade":True,
                                      "plugins.always_open_pdf_externally":True})
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)
driver.set_page_load_timeout(60)
used_rucs = []

while True:
    try:
        driver.get(url)
        used = exps.pop(0)
        wait = WebDriverWait(driver, 3)
        expedient = wait.until(EC.presence_of_element_located((By.ID,
                                         "frmBusquedaCompanias:parametroBusqueda_input")))
        expedient.send_keys(str(used))
        time.sleep(2)
        expedient.send_keys(Keys.ENTER)
        #Find the Capthca image
        wait = WebDriverWait(driver,2) #Wait element
        captcha = wait.until(EC.presence_of_element_located((By.ID,"frmBusquedaCompanias:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)

        capi = driver.find_element(By.CSS_SELECTOR,
            "input.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
        capi.send_keys(answer)
        #Continue Button Click    
        driver.find_element(By.CLASS_NAME,"ui-button-text.ui-c")
        time.sleep(1)
        break
        #Navigate to the financial information we seek.
        download(driver)
        break
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException) as e:
        print("Something Went Wrong")
        continue

In [30]:
navFin(driver)
pdfShow(driver)

Captcha Solved
Financial Info Opened Successfully
0
No audited financial statements found. Resorting to downloading the regular financials


In [66]:
# #Find the adequate information table.
# m_table = driver.find_element(By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")
# #Find the pertinent search bar with the proper search criteria.
# s_bar = m_table.find_element(By.CLASS_NAME,"ui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
# s_bar.send_keys("AUD")

if len(m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")) == 0:
    print("No audited financial statements found. Resorting to downloading the regular financials")
    s_bar.send_keys(Keys.CONTROL + "a")
    s_bar.send_keys(Keys.DELETE)
    time.sleep(5)
    s_bar.send_keys("Estado")

# rows_p = m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")
# rows_i = m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-odd")
# rows_u = zip(rows_p,rows_i)
# rows = join(rows_u)

No audited financial statements found. Resorting to downloading the regular financials


# Making it a loop for every expediente

In [10]:
not os.path.isdir("Financials\\00213")

True

In [25]:
path = r"C:\Users\00330305\Documents\Edwin\Supercias\Financials"
# path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\DataHub\Supercias\Financial Information\Financials"
url = "https://appscvsconsultas.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf"


used_rucs = []

while len(exps) != 0:
    used = exps.pop(0)
    if not os.path.isdir(f"Financials\\{used}"):
        os.mkdir(f"Financials\\{used}")       
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs",{"download.default_directory":f"{path}\\{used}",
                                      "download.prompt_for_download":False,
                                      "download.directory_upgrade":True,
                                      "plugins.always_open_pdf_externally":True})
    driver = webdriver.Chrome(options=options)
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 3)
        expedient = wait.until(EC.presence_of_element_located((By.ID,
                                         "frmBusquedaCompanias:parametroBusqueda_input")))
        expedient.send_keys(str(used))
        time.sleep(2)
        expedient.send_keys(Keys.ENTER)
        #Find the Capthca image
        wait = WebDriverWait(driver,3) #Wait element
        captcha = wait.until(EC.presence_of_element_located((By.ID,"frmBusquedaCompanias:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)

        capi = driver.find_element(By.CSS_SELECTOR,
            "input.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
        capi.send_keys(answer)
        #Continue Button Click
        driver.find_element(By.CLASS_NAME,"ui-button-text.ui-c")
        time.sleep(1)
        #Navigate to the financial information we seek.
        download(driver)
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException,StaleElementReferenceException) as e:
        print("Something Went Wrong")
        exps.insert(0,used)
        continue
driver.close()

Beginning Navigation
Captcha Solved
Financial Info Opened Successfully
Keys Sent
Navigation Finished
0
Captcha Solved
Starting Download
First Download
1
Captcha Solved
Starting Download
2
Starting Download
3
Captcha Solved
Starting Download
4
Starting Download
5
Captcha Solved
Starting Download
6
Starting Download
Load Failed...
Beginning Navigation


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x004D37D3]
	(No symbol) [0x00468B81]
	(No symbol) [0x0036B36D]
	(No symbol) [0x0034E5DA]
	(No symbol) [0x003BF0CB]
	(No symbol) [0x003D0A56]
	(No symbol) [0x003BB216]
	(No symbol) [0x00390D97]
	(No symbol) [0x0039253D]
	GetHandleVerifier [0x0074ABF2+2510930]
	GetHandleVerifier [0x00778EC1+2700065]
	GetHandleVerifier [0x0077C86C+2714828]
	GetHandleVerifier [0x00583480+645344]
	(No symbol) [0x00470FD2]
	(No symbol) [0x00476C68]
	(No symbol) [0x00476D4B]
	(No symbol) [0x00480D6B]
	(No symbol) [0x764800F9]
	RtlGetAppContainerNamedObjectPath [0x77847BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77847B8E+238]


In [13]:
navFin(driver)

Captcha Solved
Financial Info Opened Successfully


In [14]:
pdfShow(driver)

Keys Sent


In [37]:
download(driver)

Beginning Navigation


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x00D46643]
	(No symbol) [0x00CDBE21]
	(No symbol) [0x00BDDA9D]
	(No symbol) [0x00BBEF6A]
	(No symbol) [0x00C33AAB]
	(No symbol) [0x00C461B6]
	(No symbol) [0x00C2FB76]
	(No symbol) [0x00C049C1]
	(No symbol) [0x00C05E5D]
	GetHandleVerifier [0x00FBA142+2497106]
	GetHandleVerifier [0x00FE85D3+2686691]
	GetHandleVerifier [0x00FEBB9C+2700460]
	GetHandleVerifier [0x00DF3B10+635936]
	(No symbol) [0x00CE4A1F]
	(No symbol) [0x00CEA418]
	(No symbol) [0x00CEA505]
	(No symbol) [0x00CF508B]
	BaseThreadInitThunk [0x75EB6BD9+25]
	RtlGetFullPathName_UEx [0x77188FD2+1218]
	RtlGetFullPathName_UEx [0x77188F9D+1165]


In [10]:

action = ActionChains(driver)
try:
    for i,row in enumerate(rows):
        pdf = driver.find_element(By.CLASS_NAME,"ui-commandlink.ui-widget")
        driver.execute_script("arguments[0].click();",pdf)
        #Solve Captha if it pops up
        time.sleep(0.5)
        solve_other(driver)
        time.sleep(2)
        #Download the files.
        x = driver.find_element(By.ID,"dlgPresentarDocumentoPdf")
        if i == 0:
            print(i)
    #         Download Button (First Download)
            action.move_to_element_with_offset(x,370,-250)
            action.click()
            action.perform()
            time.sleep(3)
        else:
            #Download Button (After first download)
            action.move_to_element_with_offset(x,365,-210)
            action.click()
            action.perform()
       #X Button (After first download)
        action.move_to_element_with_offset(x,470,-260)
        action.click()
        action.perform()
except (ElementNotInteractableException,TimeoutException):
    driver.refresh()


0


In [5]:
#Columna random
# y = driver.find_element(By.ID,"frmInformacionCompanias:tblInformacionAnual:0:j_idt589")
#PDF
# y = driver.find_element(By.ID,"frmInformacionCompanias:tblInformacionAnual:0:j_idt599")
# y.location
#Left corner of box
x = driver.find_element(By.ID,"dlgPresentarDocumentoPdf")
print(x.location)
x_s,y_s=x.size.values()
print(y_s)
x_s -= 28
y_s = y_s/2
x_s = x_s/2
print((x_s,y_s))

#0,0 Coordinate?
# x = driver.find_element(By.TAG_NAME,"body")
# x.location
# z = driver.find_element(By.CLASS_NAME,"ui-button-icon-left.ui-icon.ui-c.btn-cerrar")
# z.click()
# x = 0
# y = 0
# z.location

{'x': 28, 'y': 0}
978
(351.5, 489.0)


In [9]:
# for i in range(1):
action = ActionChains(driver)

#Download Button
###(When using Gaming PC)
# action.move_to_element_with_offset(x,870,70)
# action.click()
# action.perform()
###When using regular PC
action.move_to_element_with_offset(x,365,-290)
action.click()
action.perform()
# #         X Button
# # time.sleep(4)
# action.move_to_element_with_offset(x,470,-280)
# action.click()
# action.perform()

#Download Button (After first download)
###When using Gaming PC
# action.move_to_element_with_offset(x,870,90)
# action.click()
# action.perform()
###When using regular PC (Reduce 70 for true regular PC. CURRENT IS FOR DHUB PC IN EDWIN'S USERNAME)
# action.move_to_element_with_offset(x,365,-270)
# action.click()
# action.perform()

#X Button (After first download)
###When Using Gaming PC
# action.move_to_element_with_offset(x,970,45)
# action.click()
# action.perform()

###When using regular PC
# action.move_to_element_with_offset(x,470,-320)
# action.click()
# action.perform()
#     action.send_keys(Keys.ENTER)
# action.click()
# action.perform()
#     action.send_keys(Keys.TAB*2)
#     action.send_keys(Keys.ENTER)
#     action.send_keys(Keys.ESCAPE)
#     action.perform()

In [ ]:
def checkDuplicates(driver):
    path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\Décimosegundo Semestre\Web Scraping Course\Financials"
    files = os.listdir(path)
    for file in files:
        check = "(" in file)
        if check == True:
            os.remove(f"{path}\\{file}")
            break
    return check

In [5]:
lookForAll(exps)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="frmInformacionCompanias:tblInformacionAnual:j_idt587:filter"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x01156643]
	(No symbol) [0x010EBE21]
	(No symbol) [0x00FEDA9D]
	(No symbol) [0x01021342]
	(No symbol) [0x0102147B]
	(No symbol) [0x01058DC2]
	(No symbol) [0x0103FDC4]
	(No symbol) [0x01056B09]
	(No symbol) [0x0103FB76]
	(No symbol) [0x010149C1]
	(No symbol) [0x01015E5D]
	GetHandleVerifier [0x013CA142+2497106]
	GetHandleVerifier [0x013F85D3+2686691]
	GetHandleVerifier [0x013FBB9C+2700460]
	GetHandleVerifier [0x01203B10+635936]
	(No symbol) [0x010F4A1F]
	(No symbol) [0x010FA418]
	(No symbol) [0x010FA505]
	(No symbol) [0x0110508B]
	BaseThreadInitThunk [0x753F00F9+25]
	RtlGetAppContainerNamedObjectPath [0x770D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x770D7B8E+238]


In [9]:
import time
start = time.time()
driver,datos = lookForAll(exps)
end = time.time()
print(f"Total iteration time = {end-start}")

,Expediente,Identificación,Nombre,Nacionalidad,Cargo,Fecha nombramiento,Periodo,Número registro mercantil,Fecha registro mercantil,RL o ADM,PDF
0,33283,101878130,PACHECO GARCIA ROMULO IVAN,ECUADOR,GERENTE GENERAL,2022-03-08,1,1458,2022-05-30,RL,NaN
1,33283,103194312,GORDILLO MONTALVAN MILTON GEOVANNI,ECUADOR,PRESIDENTE,2022-03-08,1,1457,2022-05-30,ADM,NaN
2,33283,104353701,LEON BACUILIMA JUAN CARLOS,ECUADOR,GERENTE GENERAL,2020-03-20,1,1521,2020-07-10,RL,NaN
3,33283,101878130,PACHECO GARCIA ROMULO IVAN,ECUADOR,GERENTE GENERAL,2021-03-05,1,886,2021-04-01,RL,NaN
4,33283,104353701,LEON BACUILIMA JUAN CARLOS,ECUADOR,GERENTE GENERAL,2019-03-19,1,784,2019-04-15,RL,NaN
...,...,...,...,...,...,...,...,...,...,...,...
175,33389,102285384,IÑIGUEZ SANCHEZ FELIPE JAVIER,ECUADOR,GERENTE GENERAL,2011-09-02,2,1833,2011-09-08,RL,NaN
176,33389,102285384,IÑIGUEZ SANCHEZ FELIPE JAVIER,ECUADOR,GERENTE GENERAL,2009-09-01,2,1478,2009-09-01,RL,NaN
177,33389,102285384,IÑIGUEZ SANCHEZ FELIPE JAVIER,ECUADOR,GERENTE GENERAL,2007-01-10,2,55,2007-01-10,RL,NaN
178,33389,102115904,IZQUIERDO ABAD SEBASTIAN,ECUADOR,GERENTE GENERAL,2008-12-17,2,162,2009-02-02,RL,NaN


363.6829149723053
